<h1> ¿Quien Habla? </h1>

<p> El presente proyecto identifica en de un set de datos si quien esta hablando es un politico o un medico. el objetivo es hacer esta clasificación BINARIA por medio de modelos ML. <p>
<li> NLTK: biblioteca de Python ampliamente utilizada en el procesamiento del lenguaje natural (PLN) y la lingüística computacional. Ofrece herramientas para tokenización, segmentación, etiquetado, análisis y procesamiento de texto en varios idiomas. </li>
<li> UNICODE: se utiliza para transformar texto Unicode (que incluye caracteres acentuados y caracteres especiales) en texto ASCII simple y legible.</li>

In [3]:
import string

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from unidecode import unidecode

In [6]:
dialogs = pd.read_csv('C:/Users/chech/PC Febrero 2023/1. Data Scients/Machine Learning Projects/QuienHabla/data/dialogos.csv')
dialogs.sample(5)

,Unnamed: 0,speaker,dialog,length
150734,150734,lopez-gatell,Informamos de algunos avances importantes. El ...,290
67928,67928,amlo,"Entonces, hay una carta de Porfirio que es gen...",248
14584,14584,lopez-gatell,"Por lo que se ve aquí, el porcentaje de activo...",334
71659,71659,amlo,¡Que viva Tepexi!,17
150212,150212,lopez-gatell,Lo importante es que ya llevamos 415 mil 417 v...,63


In [7]:
#Transformación de datos, cambiar la palabra "amlo" por "politico". Lo que no sea amlo dejarlo como "medico"
dialogs["speaker"] = np.where(dialogs['speaker'] == "amlo", "politico", "medico")
dialogs.sample(6)

,Unnamed: 0,speaker,dialog,length
93438,93438,politico,"No, no se modifica ninguna ley. Fue un acuerdo...",220
128477,128477,politico,Esta es la inflación anualizada.,32
80524,80524,politico,Esa era una de las reglas de la anterior polít...,143
32832,32832,medico,"Entonces, hay que quitarnos ya esta idea de: ‘...",130
102252,102252,politico,"Pero no sólo es darle la planta, porque eso no...",406
123089,123089,politico,Muy bien. Reyna.,16


<P> Metrica para el caso de estudio: ACCURACY. <p>